In [ ]:

from keras.utils import np_utils
from keras.datasets import mnist
import seaborn as sns
from keras.initializers import RandomNormal
import numpy as np
import random
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop


In [ ]:
#load dataset
(X_train, y_train), (X_test, y_test)=mnist.load_data()

In [ ]:
#chooses class 1,8,5
f=dict()
f[1]=[]
f[8]=[]
f[5]=[]
y=y_train
for i in range(60000):
  if y[i] in [1,8,5]:
    f[y[i]].append(X_train[i,:,:])

In [ ]:
#sample them into test and train set.
import random
one=random.sample(f[1],3500)
four=random.sample(f[8],1250)
five=random.sample(f[5],250)
random.shuffle(one)
random.shuffle(five)
random.shuffle(four)
X_train=one[:2800]+four[:1000]+five[:200]
X_test=one[2800:]+four[1000:]+five[200:]
y_train=[0]*2800+[1]*1000+[2]*200
y_test=[0]*700+[1]*250+[2]*50


In [ ]:
#shuffle train and test sets
train_x=np.array(X_train)
test_x=np.array(X_test)
train_y=np.array(y_train)
test_y=np.array(y_test)
shuffler1=np.random.permutation(train_y.shape[0])
train_x=train_x[shuffler1]
train_y=train_y[shuffler1]
shuffler2=np.random.permutation(test_y.shape[0])
test_x=test_x[shuffler2]
test_y=test_y[shuffler2]

In [ ]:
# Normalise the Data
train_x=train_x.reshape(train_x.shape[0],train_x.shape[1]*train_x.shape[2]).astype('float32')/255
test_x=test_x.reshape(test_x.shape[0],test_x.shape[1]*test_x.shape[2]).astype('float32')/255

In [ ]:
#converts it to one hot vector representation
Y_tr=np_utils.to_categorical(y_train,3)
Y_tt=np_utils.to_categorical(y_test,3)

In [ ]:
#model params
model=Sequential()
model.add(Dense(512,activation='relu',input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

In [ ]:
#change loss to categorical_crossentropy if needed and change weights to play with the model.
model.summary()

model.compile(loss='MSE',
              optimizer=RMSprop(),
              metrics=['accuracy','AUC'])

class_weights={0: 1,
                1:2.5,
                2: 5}
history = model.fit(train_x, Y_tr,class_weight=class_weights,
                    batch_size=64,
                    epochs=40,
                    verbose=1,
                    validation_data=(test_x, Y_tt))

In [ ]:
score = model.evaluate(test_x,Y_tt, verbose=0)
print(score)
print('Test accuracy:', score[1])

In [ ]:
#defining model again, thiis time for Focal loss.
model=Sequential()
model.add(Dense(512,activation='relu',input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

In [ ]:
#another approach Customised loss i.e. Focal Loss

from keras import backend as K

def focal_loss(y_true, y_pred):
    gamma = 2
    alpha = 0.25
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))

model.compile(loss=[focal_loss], metrics=["accuracy",'AUC'], optimizer='adam') 

history = model.fit(train_x, Y_tr,
                    batch_size=64,
                    epochs=60,
                    verbose=1,
                    validation_data=(test_x, Y_tt))

In [ ]:
score = model.evaluate(test_x,Y_tt, verbose=0)
print(score)
print('Test accuracy:', score[1])